Install the following packages

- matplotlib
- numpy
- pandas
- seaborn
- snowflake
- python
- streamlit
- plotly

In [ ]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import DataFrame, Window
from snowflake.snowpark.types import *
from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import col, sql_expr, to_timestamp
import pandas as pd

# Snowflake connection info is saved in config.py
#from config import snowflake_conn_prop


# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

________

In [ ]:
//CREATE
//OR REPLACE TABLE ONLINE_RETAIL.ONLINE_RETAIL.online_retail_new AS
//SELECT
  //online_retail.customerid, TO_TIMESTAMP (REGEXP_REPLACE (online_retail.invoicedate,'(\\d/)([0-9])(/\\d{4})','\\10\\2\\3'),'MM/DD/YYYY HH24:MI') AS invoicedate,
  //online_retail.invoiceno,
  //online_retail.stockcode,
  //online_retail.description,
  //online_retail.quantity,
  //online_retail.unitprice,
  //online_retail.country
//FROM
  //ONLINE_RETAIL.ONLINE_RETAIL.online_retail;

//DROP TABLE ONLINE_RETAIL.ONLINE_RETAIL.online_retail;
//ALTER TABLE
  //ONLINE_RETAIL.ONLINE_RETAIL.online_retail_new RENAME TO online_retail;

In [ ]:
df = session.table('ONLINE_RETAIL')
df = df.dropna()
df = df.filter("QUANTITY > 0 and UNITPRICE > 0")
df = df.filter("COUNTRY = 'United Kingdom'")
df.show()

In [ ]:
# create a new column to store the max invoicedate, use as the reference date to calculate recency
df = df.join(df.select(F.max("INVOICEDATE").alias("NOW")))

#RECENCY, FREQUENCY, MONETARY
df_tf = df.group_by("CUSTOMERID")\
          .agg(F.max("INVOICEDATE"),\
               F.max("NOW"),\
               F.sum(col("QUANTITY")*col("UNITPRICE")).alias("Monetary"),\
               F.count("INVOICENO").alias("Frequency")).select("CUSTOMERID","Frequency","Monetary",\
                                    F.datediff("days",col("MAX(INVOICEDATE)"),col("MAX(NOW)")).alias("RECENCY"))

df_tf.show()

In [ ]:
#K-means can be quite sensitive to outliers, so let’s remove them through the percentile method — any data above the 99 percentile are discarded.
freq_99pc = df_tf.select(F.percentile_cont(0.99).within_group("FREQUENCY")).collect()
rec_99pc = df_tf.select(F.percentile_cont(0.99).within_group("RECENCY")).collect()
mon_99pc = df_tf.select(F.percentile_cont(0.99).within_group("MONETARY")).collect()

df_tf = df_tf.filter((col("FREQUENCY") < freq_99pc[0][0]) & \
                     (col("RECENCY") < rec_99pc[0][0]) & \
                     (col("MONETARY") < mon_99pc[0][0]))

In [ ]:
#We can visualize the distribution of the features

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"

fig, ax = plt.subplots(1, 3, figsize=(16, 8))

df_pd = df_tf.select("CUSTOMERID", "FREQUENCY", "RECENCY", "MONETARY").toPandas()

# Recency distribution plot
sns.histplot(df_pd['RECENCY'], kde=True, ax=ax[0])

# Frequency distribution plot
sns.histplot(df_pd['FREQUENCY'], kde=True, ax=ax[1])

# Monetary distribution plot
sns.histplot(df_pd['MONETARY'], kde=True, ax=ax[2])

In [ ]:
# Implementation code for the stored procedure
def kmeans_cluster (session: Session, max_iter: int, iterate: int) -> str:
        
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    from sklearn.cluster import KMeans


    scaler = StandardScaler()
    scaler.fit(df_pd[["FREQUENCY", "RECENCY", "MONETARY"]])
    print(scaler.mean_)
    df_train = scaler.transform(df_pd[["FREQUENCY", "RECENCY", "MONETARY"]])
    print (df_train)
    inertia = []
    results = []
    

    if iterate==1:
        K = range(2,max_iter)
        for k in K:
            model = KMeans(n_clusters=k, init='k-means++', random_state=42).fit(df_train)
            inertia.append(model.inertia_)
            results = inertia
    else:
        df_pd['Cluster'] = KMeans(n_clusters=max_iter, init='k-means++', random_state=42).fit(df_train).labels_
        df_output = session.createDataFrame(df_pd)
        df_output.write.saveAsTable("RFM_Clusters",mode="overwrite")
        results = "MODEL CREATED"
        
    return results

In [ ]:
# we need to register the stored procedure with the necessary Python libraries added

from snowflake.snowpark.functions import sproc

# This adds the necessary packages to our Python environment in Snowflake
# session.clear_packages()
session.add_packages("snowflake-snowpark-python","pandas","scikit-learn")

# We need a stage to host the stored proc
_ = session.sql('CREATE STAGE IF NOT EXISTS MODELS').collect()

# Register the stored proc
session.sproc.register(kmeans_cluster, name="kmeans_cluster", is_permanent=True, stage_location="@models", replace=True)

In [ ]:
#Calling this procedure will execute the Sci-Kit Learn K-Means clustering algorithm iterating 
#over k = 2 to k = 12 on the dataset and return the inertia as an array.

max_iter = 12
results = session.call("kmeans_cluster", max_iter, 1)
inertia = np.array(results[1:-1].split(',')).astype(float)

In [ ]:
K = range(2,max_iter)
plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.title('Elbow Chart')
plt.show()

In [ ]:
#We can see that the optimal value is k = 4

max_iter = 4
results = session.call("kmeans_cluster", max_iter, 0)

In [ ]:
import streamlit as st
import snowflake.connector
from snowflake.snowpark.session import Session
import pandas as pd
import numpy as np
import plotly.express as px

# Initialize connection.
# Refer to https://docs.streamlit.io/knowledge-base/tutorials/databases/snowflake for format of secrets

# Perform query.
st.header('Customer Segmentation example with Snowpark using K-Means')

st.write('Sample cluster data that shows the recency, frequency and monetary attributes of each customer')
df = session.table("RFM_Clusters")
df_pd = df.to_pandas()
st.dataframe(df_pd)
df_pd["Cluster"] = df_pd["Cluster"].astype(str)

st.subheader('Frequency vs Recency')

fig = px.scatter(
    df_pd,
    x="FREQUENCY",
    y="RECENCY",
    color="Cluster",
    opacity=0.5,
    render_mode='svg'
)

st.plotly_chart(fig, theme="streamlit", use_container_width=True)

st.subheader('Frequency vs Monetary')

fig = px.scatter(
    df_pd,
    x="FREQUENCY",
    y="MONETARY",
    color="Cluster",
    opacity=0.5,
    render_mode='svg'
)

st.plotly_chart(fig, theme="streamlit", use_container_width=True)

st.subheader('Recency vs Monetary')

fig = px.scatter(
    df_pd,
    x="RECENCY",
    y="MONETARY",
    color="Cluster",
    opacity=0.5,
    render_mode='svg'
)

st.plotly_chart(fig, theme="streamlit", use_container_width=True)

st.markdown("**:red[Cluster 3]** are your Loyalists. They generally spend more money and more frequently.")
st.markdown("**:blue[Cluster 1]** are your Growing customers. They spend less money and less frequently, but they spent in the last 5 months.")
st.markdown("**:orange[Cluster 2]** are your Churning customers. They spend less money and less frequently, and they spent beyond the last 5 months.")
st.markdown("**Cluster** **0** sit somewhere in between.")